<a href="https://colab.research.google.com/github/hyeonji0401/CUDA_practice/blob/main/CUDA_sharedMemory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9.1 공유 메모리 사용 방법
- 크게 다음 3가지로 구분할 수 있음
  - 스레드 블록 내 스레드들의 공유 데이터 보관
  - L1 캐시(HW 관리 캐시)
  - 사용자 관리(user-managed) 캐시

1.스레드 블록 내 스레드들의 공유 데이터 보관
- 만약 블록 내 스레드들 사이에 정보 또는 데이터를 주고 받아야한다면 공유 메모리에 변수 등 공간을 만들고 여러 스레드들이 해당 공간에 데이터를 읽고 쓰는 형태로 구현 가능함
- 동기화를 주의해야함
  - 공유 메모리는 여러 스레드가 동시에 데이터를 쓰거나 읽을 수 있기 때문에 읽고 쓰는 순서에 대한 약속을 지켜야함
  - 동기화란 스레드들 사이에 순서를 맞추는 행위 또는 서로 정보를 교환하는 행위라고 말할 수 있음
  - 볼록 내 스레드들에 대한 동기화함수는 _syncthreads()
    - 블록 내 모든 스레드가 해당 지점에 도착할 때까지 다른 스레드들의 진행을 막는 역할을 함